In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.neighbors import KNeighborsClassifier 
sns.set_style("whitegrid")
sns.set_context("poster")
#read in df
ppdf=pd.read_pickle('ready_for_ml.pkl')
sns.set()

Because the descriptive stats did not show very much correlation we are going to use a Lasso regularized regression in order to discover which variables are strong predictors of errors in the point spread and O/U lines. First we need to seperate what features we want to use. We will use a few different cmbinations of features to see which one best predicts variability.

In [20]:
#Create Numpy arrays from our data to pass to ml algorithms 
ySpread=np.reshape(np.array(ppdf.loc[:,'results-spread']),(-1,1))
yOver=np.reshape(np.array(ppdf.loc[:,'over_diff']),(-1,1))
ySpreadabs=np.reshape(np.array(ppdf.loc[:,'resultmSpreadabs']),(-1,1))
yOverabs=np.reshape(np.array(ppdf.loc[:,'over_diffabs']),(-1,1))
Xweath= ppdf.loc[:, ['weather_temperature','weather_wind_mph', 'precipitation']]

Xweath=np.array(Xweath)
Xrank=np.reshape(np.array(ppdf.weather_rank),(-1,1))
Xbw=np.reshape(np.array(ppdf.bw.astype(int)),(-1,1))
Xrank

array([[2],
       [2],
       [2],
       ...,
       [1],
       [1],
       [1]])

In [21]:
#Examine coeficients of various relationships to see if they should be studied further
lass= Lasso(alpha=.3,normalize=True)
lass.fit(Xweath, ySpread)
lass.coef_

array([ 0., -0.,  0.])

In [22]:
lass= Lasso(alpha=.3,normalize=True)
lass.fit(Xweath, yOver)
lass.coef_

array([-0., -0., -0.])

In [23]:
lass= Lasso(alpha=.3,normalize=True)
lass.fit(Xweath, yOverabs)
lass.coef_

array([-0.,  0.,  0.])

In [24]:
lass= Lasso(alpha=.3,normalize=True)
lass.fit(Xweath, ySpreadabs)
lass.coef_

array([ 0., -0., -0.])

In [35]:
reg= LinearRegression()
cv= cross_val_score(reg, Xrank, ySpread, cv=5)
cv.mean()

-0.0012249360959593724

In [36]:
reg= LinearRegression()
cv= cross_val_score(reg, Xrank, ySpreadabs, cv=5)
cv.mean()

-0.0021082754669870417

In [37]:
reg= LinearRegression()
cv= cross_val_score(reg, Xrank, yOver, cv=5)
cv.mean()

-0.0011041408928526674

In [38]:

reg= LinearRegression()
cv= cross_val_score(reg, Xrank, yOverabs, cv=5)
cv.mean()

-0.00017931853688866184

In [39]:
reg= LinearRegression()
cv= cross_val_score(reg, Xbw, ySpread, cv=5)
cv.mean()

-0.0010300868315448186

In [40]:

reg= LinearRegression()
cv= cross_val_score(reg, Xbw, yOver, cv=5)
cv.mean()


-0.0001850572053200494

In [41]:
reg= LinearRegression()
cv= cross_val_score(reg, Xbw, ySpreadabs, cv=5)
cv.mean()

-0.0026912527065614976

In [42]:
reg= LinearRegression()
cv= cross_val_score(reg, Xbw, yOverabs, cv=5)
cv.mean()

-0.0006511062517112531

It appears the most significant effects are the effects of bw and rank on the over difference. While thees effects are negitive we should investigate further to see what amount of the variance can be explained by weather. As you can see the r-squares were negitive. A negitve r-squared means a horizontal line would better fit the data so the explained variance is zero.

With this model being somewhat of a failiure, I think it would be intersting to see if the model can predict the data in reverse. While this is not useful I find it interesting.

In [18]:
ybw=Xbw.reshape(-1,1)
XSpreadOver=ppdf.loc[:,['results-spread','over_diff']]

XSpreadOver=np.array(XSpreadOver.values)
X_train, X_test, y_train, y_test = train_test_split(XSpreadOver, ybw, test_size = .2)
neighbor= np.arange(1,9)
training_score=np.empty(len(neighbor))
test_score=np.empty(len(neighbor))
for i,k in enumerate(neighbor):
    knn=KNeighborsClassifier(n_neighbors= k)
    
    knn.fit(X_train, y_train.ravel())
    training_score[i] = knn.score(X_train, y_train)

    
    test_score[i] = knn.score(X_test, y_test)
    
print(test_score)
print('-----------')
print(training_score)


[0.7620339  0.85220339 0.82847458 0.85627119 0.8440678  0.85830508
 0.85694915 0.86169492]
-----------
[0.93915254 0.87694915 0.88       0.86559322 0.86644068 0.86423729
 0.86338983 0.86288136]


Interestingly, It appears the model is able to predict, with about 85% accuracy wheather or not a game was played in bad weather simply by the difference between the predicted spread/Over/Under total and the actual score of the game. This may mean their is some unseen correlation. These results do not seem to be actionable because the model cannaot predict future matchups because it does not know the final score. Additionally, if the final score was known before the matchup the model would already be useless.